<a href="https://colab.research.google.com/github/saigirishgilly98/Major_Project/blob/main/ResumeParserCareerPathRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Reference : https://omkarpathak.in/2018/12/18/writing-your-own-resume-parser/

In [1]:
! pip install pdfminer.six

     |████████████████████████████████| 5.6MB 2.7MB/s 
     |████████████████████████████████| 2.6MB 44.7MB/s 


In [4]:
import io

In [6]:
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as fh:
        # iterate over all pages of PDF document
        for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
            # creating a resoure manager
            resource_manager = PDFResourceManager()
            
            # create a file handle
            fake_file_handle = io.StringIO()
            
            # creating a text converter object
            converter = TextConverter(
                                resource_manager, 
                                fake_file_handle, 
                                codec='utf-8', 
                                laparams=LAParams()
                        )

            # creating a page interpreter
            page_interpreter = PDFPageInterpreter(
                                resource_manager, 
                                converter
                            )

            # process current page
            page_interpreter.process_page(page)
            
            # extract text
            text = fake_file_handle.getvalue()
            yield text

            # close open handles
            converter.close()
            fake_file_handle.close()

# calling above function and extracting text
text = ''
for page in extract_text_from_pdf('Abhishak_Resume.pdf'):
    text += ' ' + page



In [7]:
text

' R abhishakvarshney@gmail.com\n\nﬂ linkedin.com/in/abhishakvarshney\n\n(cid:135) github.com/abhishakvarshney\n\nABHISHAK VARSHNEY\n\n(cid:211) +91-8433489919\n(cid:7) live:abhishakvarshney\n\nEDUCATION\n\nNIT Jaipur(MNIT/MREC)\nB.Tech - Metallurgical & Materials Engineering. CGPA: 7.5\n\n(cid:17) 2014 – 2018\nB.B.S.S.M. Inter College\nIntermediate/+2; Uttar-Pradesh Board Result: 91.00%\n\n(cid:17) 2012 – 2013\nB.B.S.S.M. Inter College\nHigh-School; Uttar-Pradesh Board Result: 72.83%\n\n(cid:17) 2010 – 2011\n\nEXPERIENCE\n\nSoftware Engineer - Analytics\nSkilrock Technologies | Sugal & Damani Group\n\n(cid:17) June 2018 – Present\n\n‰ Gurugram,India\n\nNLP - ChatBot (ARENA)\n• Developed an NLP based chatbot in Python from that users can\nplay games, purchase tickets & can chat small talk with user.\n• Deployed on Skilrock Technologies Client Gaming and Lottery\nEngine website, Android and iOS Apps and on Facebook Mes-\nsenger.\n\n• Technology: Python | Rasa, Microsoft Bot Framework\n\n

In [8]:
! pip install spacy
! python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


## Extracting Name

In [11]:
import spacy
from spacy.matcher import Matcher

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)

def extract_name(resume_text):
    nlp_text = nlp(resume_text)
    
    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', None, pattern)
    
    matches = matcher(nlp_text)
    
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        print(span.text)

In [ ]:
extract_name(text)

## Extracting Phone Numbers

In [13]:
import re

def extract_mobile_number(text):
    flag = 0
    phone = re.findall(re.compile(r'(\d{3}-\d{3}-\d{4})'), text)
    if len(phone) == 0:
        phone = re.findall(re.compile(r'(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?'), text)
        flag = 1
    print(phone)
    if phone:
        if flag == 1:
            number = ''.join(phone[0])
        else:
            number = ''.join(phone[0].split('-'))
        if len(number) > 10:
            return '+' + number
        else:
            return number

In [14]:
mobile_number = extract_mobile_number(text)
print(mobile_number)

[('91', '', '84', '334', '8991', '')]
+91843348991


## Extracting E-Mail

In [15]:
import re

def extract_email(email):
    email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)", email)
    if email:
        try:
            return email[0].split()[0].strip(';')
        except IndexError:
            return None

In [16]:
email = extract_email(text)
print(email)

abhishakvarshney@gmail.com


## Extracting Skills

In [17]:
import pandas as pd

In [19]:
skills_dataset = pd.read_csv('linkedin_skill.txt', sep='\n', header=None)
skills_dataset.columns = ['skill']
skills_dataset.head()

,skill
0,(ISC)2
1,.NET
2,.NET CLR
3,.NET Compact Framework
4,.NET Framework


In [20]:
skills = []
for i in range(len(skills_dataset)):
    skills.append(str(skills_dataset.loc[i, 'skill']).lower())
skills[-20:]

['zoominfo',
 'zoomtext',
 'zoomerang',
 'zope',
 'zotero',
 'zsh',
 'zuken',
 'zultys',
 'zulu education products',
 'zuludesk',
 'zumba',
 'zumba instruction',
 'zuora',
 'zymography',
 'zynx',
 'zyxel',
 'z/os',
 'z/vm',
 'zlinux',
 'zseries']

In [21]:
import spacy

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

def extract_skills(resume_text):
    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    skillset = []
    
    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
    
    # check for bi-grams and tri-grams (example: machine learning)
    for token in nlp_text.noun_chunks:
        token = token.text.lower().strip()
        if token in skills:
            skillset.append(token)
    
    return [i.capitalize() for i in set([i.lower() for i in skillset])]

In [22]:
skills = extract_skills(text)
print(skills)

['Learning', 'Mass balance', 'Coursera', 'Mysql', 'Arima', 'Hobbies', 'Opencv', 'Linux', 'Google', 'Twitter', 'Skating', 'Terminal', 'Machine learning', 'Keras', 'Analytics', 'Forest', 'Nlp', 'Languages', 'Framework', 'Excel', 'Mongodb inc.', 'Robotics', 'Microsoft', 'Git', 'Horses', 'Steel', 'Mining', 'Features', 'Live', 'Prediction', 'Materials', 'Application', 'R', 'Facebook', 'Apps', 'Processing', 'Visualisation', 'Database', 'Android', 'Dogs', 'Balance', 'Small talk', 'Sql', 'Text mining', 'Profiles', 'Cd', 'Object detection', 'Gaming', 'Bot', 'Ios', 'Mv', 'Olympic', 'Cp', 'Games', 'Python', 'Image processing', 'Google analytics', 'Technology', 'Mongodb', 'Chat', 'Nginx', 'Heat']


## Extracting Education

In [24]:
import re
import spacy
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# Grad all general stop words
STOPWORDS = set(stopwords.words('english'))

# Education Degrees
EDUCATION = [
            'BE','B.E.', 'B.E', 'BS', 'B.S', 
            'ME', 'M.E', 'M.E.', 'MS', 'M.S', 
            'BTECH', 'B.TECH', 'M.TECH', 'MTECH', 
            'SSC', 'HSC', 'CBSE', 'ICSE', 'XII', 'X', 'PHD'
        ]

def extract_education(resume_text):
    nlp_text = nlp(resume_text)

    # Sentence Tokenizer
    nlp_text = [sent.string.strip() for sent in nlp_text.sents]
    edu = {}
    # Extract education degree
    for index, text in enumerate(nlp_text):
        for tex in text.split():
            # Replace all special symbols
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            if tex.upper() in EDUCATION and tex not in STOPWORDS:
                try:
                    if index + 1 < len(nlp_text):
                        if tex in edu:
                            edu[tex] += text + nlp_text[index + 1]
                        else:
                            edu[tex] = text + nlp_text[index + 1]
                    if index - 1 < len(nlp_text):
                        if tex in edu:
                            edu[tex] += nlp_text[index - 1]
                        else:
                            edu[tex] = nlp_text[index - 1]
                    if index + 2 < len(nlp_text):
                        if tex in edu:
                            edu[tex] += nlp_text[index + 2]
                        else:
                            edu[tex] = nlp_text[index + 2]
                    if index - 2 < len(nlp_text):
                        if tex in edu:
                            edu[tex] += nlp_text[index - 2] 
                        else:
                            edu[tex] = nlp_text[index - 2]
                    if index + 3 < len(nlp_text):
                        if tex in edu:
                            edu[tex] += nlp_text[index + 3] 
                        else:
                            edu[tex] = nlp_text[index + 3]
                    if index - 3 < len(nlp_text):
                        if tex in edu:
                            edu[tex] += nlp_text[index - 3]
                        else:
                            edu[tex] = nlp_text[index - 3]
                    if index - 4 < len(nlp_text):
                        if tex in edu:
                            edu[tex] += nlp_text[index - 4]
                        else:
                            edu[tex] = nlp_text[index - 4]
                except:
                    edu[tex] = text
    print(edu)
    # Extract year
    education = []
    for key in edu.keys():
        year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])
        if year:
            education.append((key, ''.join(year[0])))
        else:
            education.append(key)
    return education

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [25]:
education = extract_education(text)
print(education)

{'BTech': 'B.Tech - Metallurgical & Materials Engineering.CGPA:NIT Jaipur(MNIT/MREC)7.5EDUCATION(cid:17) 2014 – 2018abhishakvarshney(cid:7) live:', 'MS': '• Technology: MS –Excel• Analyzed and Balanced Heat and Mass data from Raw Material\n\nto liquid steel making process i.e. from raw to production.CERTIFICATIONJamshedpur,India\n\nHeat & Mass Balance in BOF Vessel• Machine Learning: Coursera- July 2017\n\n‰May 2017'}
[('BTech', '2014'), ('MS', '2017')]


In [26]:
result = {'education': education, 'email': email, 'mobile_number': mobile_number, 'skills': skills}
print(result)

{'education': [('BTech', '2014'), ('MS', '2017')], 'email': 'abhishakvarshney@gmail.com', 'mobile_number': '+91843348991', 'skills': ['Learning', 'Mass balance', 'Coursera', 'Mysql', 'Arima', 'Hobbies', 'Opencv', 'Linux', 'Google', 'Twitter', 'Skating', 'Terminal', 'Machine learning', 'Keras', 'Analytics', 'Forest', 'Nlp', 'Languages', 'Framework', 'Excel', 'Mongodb inc.', 'Robotics', 'Microsoft', 'Git', 'Horses', 'Steel', 'Mining', 'Features', 'Live', 'Prediction', 'Materials', 'Application', 'R', 'Facebook', 'Apps', 'Processing', 'Visualisation', 'Database', 'Android', 'Dogs', 'Balance', 'Small talk', 'Sql', 'Text mining', 'Profiles', 'Cd', 'Object detection', 'Gaming', 'Bot', 'Ios', 'Mv', 'Olympic', 'Cp', 'Games', 'Python', 'Image processing', 'Google analytics', 'Technology', 'Mongodb', 'Chat', 'Nginx', 'Heat']}
